In [3]:
#installing the dependencies
!pip install demucs
!pip install soundfile
!pip install librosa

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with 

In [4]:
#importing libraries
import os
import soundfile
import librosa
import glob
import tqdm

In [5]:
#setting the paths
songs_path = "/content/drive/MyDrive/songs"
vocals_path = "/content/drive/MyDrive/vocals"
# Create directories if they do not exist
if not os.path.exists(songs_path):
    os.makedirs(songs_path)

if not os.path.exists(vocals_path):
    os.makedirs(vocals_path)

In [6]:
#extracting vocal
ctr = 0
model = "mdx_extra_q"
for audio_file in tqdm.tqdm(os.listdir(songs_path)):
    !demucs -n mdx_extra_q --two-stems=vocals "{os.path.join(songs_path, audio_file)}" -o {vocals_path}
    vocal_name=f"vocal_{ctr}.wav"
    song_name = os.path.splitext(audio_file)[0]

    !mv "{os.path.join(vocals_path, model, song_name, 'vocals.wav')}" "{os.path.join(vocals_path, vocal_name)}"
    !rm -r "{os.path.join(vocals_path, model, song_name)}/"

    ctr += 1

  0%|          | 0/6 [00:00<?, ?it/s]

FATAL: Trying to use DiffQ, but diffq is not installed.
On Windows run: python.exe -m pip install diffq 
On Linux/Mac, run: python3 -m pip install diffq
mv: cannot stat '/content/drive/MyDrive/vocals/mdx_extra_q/7ddf-b38b-40ed-8fca-641207d1938b/vocals.wav': No such file or directory
rm: cannot remove '/content/drive/MyDrive/vocals/mdx_extra_q/7ddf-b38b-40ed-8fca-641207d1938b/': No such file or directory


 17%|█▋        | 1/6 [00:03<00:16,  3.22s/it]

FATAL: Trying to use DiffQ, but diffq is not installed.
On Windows run: python.exe -m pip install diffq 
On Linux/Mac, run: python3 -m pip install diffq
mv: cannot stat '/content/drive/MyDrive/vocals/mdx_extra_q/a774-e209-4af3-a0c2-5930bb35f8b6/vocals.wav': No such file or directory
rm: cannot remove '/content/drive/MyDrive/vocals/mdx_extra_q/a774-e209-4af3-a0c2-5930bb35f8b6/': No such file or directory


 33%|███▎      | 2/6 [00:05<00:10,  2.52s/it]

...

 67%|██████▋   | 4/6 [00:09<00:04,  2.34s/it]

FATAL: Trying to use DiffQ, but diffq is not installed.
On Windows run: python.exe -m pip install diffq 
On Linux/Mac, run: python3 -m pip install diffq
mv: cannot stat '/content/drive/MyDrive/vocals/mdx_extra_q/.ipynb_checkpoints/vocals.wav': No such file or directory
rm: cannot remove '/content/drive/MyDrive/vocals/mdx_extra_q/.ipynb_checkpoints/': No such file or directory


 83%|████████▎ | 5/6 [00:11<00:02,  2.19s/it]

FATAL: Trying to use DiffQ, but diffq is not installed.
On Windows run: python.exe -m pip install diffq 
On Linux/Mac, run: python3 -m pip install diffq
mv: cannot stat '/content/drive/MyDrive/vocals/mdx_extra_q/facf-d04c-45a8-ad4e-4dc200921c99/vocals.wav': No such file or directory
rm: cannot remove '/content/drive/MyDrive/vocals/mdx_extra_q/facf-d04c-45a8-ad4e-4dc200921c99/': No such file or directory


100%|██████████| 6/6 [00:13<00:00,  2.29s/it]


In [7]:
#removing the silence
import numpy as np

def get_rms(
    y,
    *,
    frame_length=2048,
    hop_length=512,
    pad_mode="constant",
):
    padding = (int(frame_length // 2), int(frame_length // 2))
    y = np.pad(y, padding, mode=pad_mode)

    axis = -1
    # put our new within-frame axis at the end for now
    out_strides = y.strides + tuple([y.strides[axis]])
    # Reduce the shape on the framing axis
    x_shape_trimmed = list(y.shape)
    x_shape_trimmed[axis] -= frame_length - 1
    out_shape = tuple(x_shape_trimmed) + tuple([frame_length])
    xw = np.lib.stride_tricks.as_strided(
        y, shape=out_shape, strides=out_strides
    )
    if axis < 0:
        target_axis = axis - 1
    else:
        target_axis = axis + 1
    xw = np.moveaxis(xw, -1, target_axis)
    # Downsample along the target axis
    slices = [slice(None)] * xw.ndim
    slices[axis] = slice(0, None, hop_length)
    x = xw[tuple(slices)]

    # Calculate power
    power = np.mean(np.abs(x) ** 2, axis=-2, keepdims=True)

    return np.sqrt(power)


class Slicer:
    def __init__(self,
                 sr: int,
                 threshold: float = -40.,
                 min_length: int = 5000,
                 min_interval: int = 300,
                 hop_size: int = 20,
                 max_sil_kept: int = 5000):
        if not min_length >= min_interval >= hop_size:
            raise ValueError('The following condition must be satisfied: min_length >= min_interval >= hop_size')
        if not max_sil_kept >= hop_size:
            raise ValueError('The following condition must be satisfied: max_sil_kept >= hop_size')
        min_interval = sr * min_interval / 1000
        self.threshold = 10 ** (threshold / 20.)
        self.hop_size = round(sr * hop_size / 1000)
        self.win_size = min(round(min_interval), 4 * self.hop_size)
        self.min_length = round(sr * min_length / 1000 / self.hop_size)
        self.min_interval = round(min_interval / self.hop_size)
        self.max_sil_kept = round(sr * max_sil_kept / 1000 / self.hop_size)

    def _apply_slice(self, waveform, begin, end):
        if len(waveform.shape) > 1:
            return waveform[:, begin * self.hop_size: min(waveform.shape[1], end * self.hop_size)]
        else:
            return waveform[begin * self.hop_size: min(waveform.shape[0], end * self.hop_size)]

    def slice(self, waveform):
        if len(waveform.shape) > 1:
            samples = waveform.mean(axis=0)
        else:
            samples = waveform
        if samples.shape[0] <= self.min_length:
            return [waveform]
        # if samples.shape[0]//self.hop_size <= self.min_length:
        #     return [waveform]

        rms_list = get_rms(y=samples, frame_length=self.win_size, hop_length=self.hop_size).squeeze(0)
        sil_tags = []
        silence_start = None
        clip_start = 0
        for i, rms in enumerate(rms_list):
            # Keep looping while frame is silent.
            if rms < self.threshold:
                # Record start of silent frames.
                if silence_start is None:
                    silence_start = i
                continue
            # Keep looping while frame is not silent and silence start has not been recorded.
            if silence_start is None:
                continue
            # Clear recorded silence start if interval is not enough or clip is too short
            is_leading_silence = silence_start == 0 and i > self.max_sil_kept
            need_slice_middle = i - silence_start >= self.min_interval and i - clip_start >= self.min_length
            if not is_leading_silence and not need_slice_middle:
                silence_start = None
                continue
            # Need slicing. Record the range of silent frames to be removed.
            if i - silence_start <= self.max_sil_kept:
                pos = rms_list[silence_start: i + 1].argmin() + silence_start
                if silence_start == 0:
                    sil_tags.append((0, pos))
                else:
                    sil_tags.append((pos, pos))
                clip_start = pos
            elif i - silence_start <= self.max_sil_kept * 2:
                pos = rms_list[i - self.max_sil_kept: silence_start + self.max_sil_kept + 1].argmin()
                pos += i - self.max_sil_kept
                pos_l = rms_list[silence_start: silence_start + self.max_sil_kept + 1].argmin() + silence_start
                pos_r = rms_list[i - self.max_sil_kept: i + 1].argmin() + i - self.max_sil_kept
                if silence_start == 0:
                    sil_tags.append((0, pos_r))
                    clip_start = pos_r
                else:
                    sil_tags.append((min(pos_l, pos), max(pos_r, pos)))
                    clip_start = max(pos_r, pos)
            else:
                pos_l = rms_list[silence_start: silence_start + self.max_sil_kept + 1].argmin() + silence_start
                pos_r = rms_list[i - self.max_sil_kept: i + 1].argmin() + i - self.max_sil_kept
                if silence_start == 0:
                    sil_tags.append((0, pos_r))
                else:
                    sil_tags.append((pos_l, pos_r))
                clip_start = pos_r
            silence_start = None
        # Deal with trailing silence.
        total_frames = rms_list.shape[0]
        if silence_start is not None and total_frames - silence_start >= self.min_interval:
            silence_end = min(total_frames, silence_start + self.max_sil_kept)
            pos = rms_list[silence_start: silence_end + 1].argmin() + silence_start
            sil_tags.append((pos, total_frames + 1))
        # Apply and return slices.
        if len(sil_tags) == 0:
            return [waveform]
        else:
            chunks = []
            if sil_tags[0][0] > 0:
                chunks.append(self._apply_slice(waveform, 0, sil_tags[0][0]))
            for i in range(len(sil_tags) - 1):
                chunks.append(self._apply_slice(waveform, sil_tags[i][1], sil_tags[i + 1][0]))
            if sil_tags[-1][1] < total_frames:
                chunks.append(self._apply_slice(waveform, sil_tags[-1][1], total_frames))
            return chunks

In [8]:
#remove the silence from audio and split at that silence

non_silence_audio = "/content/drive/MyDrive/non_silenced/"
if not os.path.exists(non_silence_audio):
    os.makedirs(non_silence_audio)

for audio_path in os.listdir(vocals_path):
    print(audio_path)
    audio_path = os.path.join(vocals_path, audio_path)
    if len(os.path.basename(audio_path).split(".")) == 1:
      continue
    audio_name = os.path.basename(audio_path).split(".")[0]
    audio, sr = librosa.load(audio_path, sr=None, mono=False)
    slicer = Slicer(
        sr=sr,
        threshold=-25,
        min_length=7000,
        min_interval=600,
        hop_size=15,
        max_sil_kept=700
    )
    chunks = slicer.slice(audio)
    for i, chunk in enumerate(chunks):
        if len(chunk.shape) > 1:
            chunk = chunk.T  # Swap axes if the audio is stereo.

        print("\twrote: "+os.path.join(non_silence_audio,f'{audio_name}_{i}.wav'))
        soundfile.write(os.path.join(non_silence_audio,f'{audio_name}_{i}.wav'), chunk, sr)  # Save sliced audio files with soundfile.


7ddf-b38b-40ed-8fca-641207d1938b.mp3
	wrote: /content/drive/MyDrive/non_silenced/7ddf-b38b-40ed-8fca-641207d1938b_0.wav
	wrote: /content/drive/MyDrive/non_silenced/7ddf-b38b-40ed-8fca-641207d1938b_1.wav
	wrote: /content/drive/MyDrive/non_silenced/7ddf-b38b-40ed-8fca-641207d1938b_2.wav
	wrote: /content/drive/MyDrive/non_silenced/7ddf-b38b-40ed-8fca-641207d1938b_3.wav
	wrote: /content/drive/MyDrive/non_silenced/7ddf-b38b-40ed-8fca-641207d1938b_4.wav
	wrote: /content/drive/MyDrive/non_silenced/7ddf-b38b-40ed-8fca-641207d1938b_5.wav
	wrote: /content/drive/MyDrive/non_silenced/7ddf-b38b-40ed-8fca-641207d1938b_6.wav
a774-e209-4af3-a0c2-5930bb35f8b6.mp3
	wrote: /content/drive/MyDrive/non_silenced/a774-e209-4af3-a0c2-5930bb35f8b6_0.wav
	wrote: /content/drive/MyDrive/non_silenced/a774-e209-4af3-a0c2-5930bb35f8b6_1.wav
	wrote: /content/drive/MyDrive/non_silenced/a774-e209-4af3-a0c2-5930bb35f8b6_2.wav
	wrote: /content/drive/MyDrive/non_silenced/a774-e209-4af3-a0c2-5930bb35f8b6_3.wav
	wrote: /cont

/usr/local/lib/python3.10/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/vocals/.ipynb_checkpoints'

In [9]:
#split audios in clips of 6 seconds for training
from tqdm import tqdm
# set parameters
min_length = 6     # minimum length of each segment in seconds
sliced_dir = "/content/drive/MyDrive/sliced/"
# create output directory if it doesn't exist
if not os.path.exists(sliced_dir):
    os.makedirs(sliced_dir)

print(f"slicing audios to clips of {min_length}seconds")
cnt = 0
# loop through all .mp3 files in original_audio directory
for file in tqdm(os.listdir(non_silence_audio)):
    if file.endswith('.wav'):
        # load audio file
        audio_name = file.split('.')[0]
        audio_path = os.path.join(non_silence_audio, file)
        audio, sr = librosa.load(audio_path)

        min_samples = int(min_length * sr)

        # Split the audio into segments
        segments = []
        segment_start = 0
        while segment_start + min_samples < len(audio):
            segment_end = segment_start + min_samples
            segments.append(audio[segment_start:segment_end])
            segment_start = segment_end

        # Write each segment to a separate file
        for i, segment in enumerate(segments):
            segment_path = os.path.join(sliced_dir,f'segments_{audio_name}_{cnt}.wav')
            cnt += 1
            soundfile.write(segment_path, segment, sr)



slicing audios to clips of 6seconds


100%|██████████| 28/28 [00:10<00:00,  2.55it/s]


In [10]:
#pretrained models
import os
if not os.path.exists("/content/drive/MyDrive/sovits4data/logs/44k"):
  os.makedirs("/content/drive/MyDrive/sovits4data/logs/44k")

!wget https://huggingface.co/justinjohn-03/so-vits-svc-4.0-v2-pretrained/resolve/main/D_0.pth -P /content/drive/MyDrive/sovits4data/logs/44k
!wget https://huggingface.co/justinjohn-03/so-vits-svc-4.0-v2-pretrained/resolve/main/G_0.pth -P /content/drive/MyDrive/sovits4data/logs/44k

--2024-05-19 13:44:32--  https://huggingface.co/justinjohn-03/so-vits-svc-4.0-v2-pretrained/resolve/main/D_0.pth
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.17, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/28/64/2864b275cb5e875d4c3f832b547244a678d68bec7ac9aa4098a5b93f356b9c2e/f6d64c1423f991b23ca682a6232b5fba7ee6c84caa22a88c8f9aeedfe9b501a1?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27D_0.pth%3B+filename%3D%22D_0.pth%22%3B&Expires=1716385472&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjM4NTQ3Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8yOC82NC8yODY0YjI3NWNiNWU4NzVkNGMzZjgzMmI1NDcyNDRhNjc4ZDY4YmVjN2FjOWFhNDA5OGE1YjkzZjM1NmI5YzJlL2Y2ZDY0YzE0MjNmOTkxYjIzY2E2ODJhNjIzMmI1ZmJhN2VlNmM4NGNhYTIyYTg4YzhmOWFlZWRmZTliNTAxYTE%7EcmV

In [11]:
#cloning repo and installing dependencies
!git clone https://github.com/svc-develop-team/so-vits-svc -b 4.0
!pip uninstall -y torchdata torchtext
!pip install --upgrade pip setuptools numpy numba
!pip install pyworld praat-parselmouth fairseq tensorboardX torchcrepe librosa==0.9.1
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

%cd /content/so-vits-svc

!curl -L https://github.com/openvpi/vocoders/releases/download/nsf-hifigan-v1/nsf_hifigan_20221211.zip -o /content/so-vits-svc/nsf_hifigan_20221211.zip
!unzip nsf_hifigan_20221211.zip
!rm -rf pretrain/nsf_hifigan
!mv -v nsf_hifigan pretrain

!curl -L https://ibm.ent.box.com/shared/static/z1wgl1stco8ffooyatzdwsqn2psd9lrr -o /content/so-vits-svc/hubert/checkpoint_best_legacy_500.pt

Cloning into 'so-vits-svc'...
fatal: Remote branch 4.0 not found in upstream origin
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 wil

In [2]:
#writing files and paths
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"

RAW_DIR = sovits_data_dir + "raw/"
RESULTS_DIR = sovits_data_dir + "results/"
FILELISTS_DIR = sovits_data_dir + "filelists/"
CONFIGS_DIR = sovits_data_dir + "configs/"
LOGS_DIR = sovits_data_dir + "logs/44k/"
config_path = "configs/config.json"
sync_raw_and_results = True
if sync_raw_and_results:
  !mkdir -p {RAW_DIR}
  !mkdir -p {RESULTS_DIR}
  !rm -rf /content/so-vits-svc/raw
  !rm -rf /content/so-vits-svc/results
  !ln -s {RAW_DIR} /content/so-vits-svc/raw
  !ln -s {RESULTS_DIR} /content/so-vits-svc/results


sync_configs_and_logs = True
if sync_configs_and_logs:
  !mkdir -p {FILELISTS_DIR}
  !mkdir -p {CONFIGS_DIR}
  !mkdir -p {LOGS_DIR}
  !rm -rf /content/so-vits-svc/filelists
  !rm -rf /content/so-vits-svc/configs
  !rm -rf /content/so-vits-svc/logs/44k
  !ln -s {FILELISTS_DIR} /content/so-vits-svc/filelists
  !ln -s {CONFIGS_DIR} /content/so-vits-svc/configs
  !ln -s {LOGS_DIR} /content/so-vits-svc/logs/44k

ln: failed to create symbolic link '/content/so-vits-svc/raw': No such file or directory
ln: failed to create symbolic link '/content/so-vits-svc/results': No such file or directory
ln: failed to create symbolic link '/content/so-vits-svc/filelists': No such file or directory
ln: failed to create symbolic link '/content/so-vits-svc/configs': No such file or directory
ln: failed to create symbolic link '/content/so-vits-svc/logs/44k': No such file or directory


In [ ]:
# link dataset in dataset_raw
!ln -s /content/drive/MyDrive/sliced/ /content/so-vits-svc/dataset_raw/

In [ ]:
#resampling
%cd /content/so-vits-svc
!python resample.py

In [ ]:
#generating config
%cd /content/so-vits-svc

# speech_encoder = "vec768l12"

# from pretrain.meta import download_dict
# download_dict = download_dict()

# url = download_dict[speech_encoder]["url"]
# output = download_dict[speech_encoder]["output"]

# import os
# if not os.path.exists(output):
#   !curl -L {url} -o {output}

!python preprocess_flist_config.py #--speech_encoder={speech_encoder}

In [ ]:
#genrating hubert
%cd /content/so-vits-svc
!python preprocess_hubert_f0.py

In [ ]:
#saving
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"
!mkdir -p {sovits_data_dir}
!zip -r dataset.zip /content/so-vits-svc/dataset
!cp -vr dataset.zip "{sovits_data_dir}"

In [ ]:
#loading preprocessed
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"
CONFIG = sovits_data_dir + "configs/"
FILELISTS = sovits_data_dir + "filelists/"
DATASET = sovits_data_dir + "dataset.zip"

!cp -vr {CONFIG} /content/so-vits-svc/
!cp -vr {FILELISTS} /content/so-vits-svc/
!unzip {DATASET} -d /

In [ ]:
#train
tensorboard_on = False
if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k
%cd /content/so-vits-svc
!python train.py -c configs/config.json -m 44k

In [ ]:
%cd /content/so-vits-svc
!python cluster/train_cluster.py

In [ ]:
#infer
wav_filename = "cheque.wav"
model_filename = "G_128800.pth"
model_path = "/content/so-vits-svc/logs/44k/" + model_filename
speaker = "sliced"
trans = "0"
cluster_infer_ratio = "0"
auto_predict_f0 = False
apf = ""
if auto_predict_f0:
  apf = " -a "
f0_mean_pooling = False
fmp = ""
if f0_mean_pooling:
  fmp = " -fmp "

enhance = False
ehc = ""
if enhance:
  ehc = " -eh "

config_filename = "config.json"
config_path = "/content/so-vits-svc/configs/" + config_filename
kmeans_filenname = "kmeans_10000.pt"
kmeans_path = "/content/so-vits-svc/logs/44k/" + kmeans_filenname
slice_db = "-40"
wav_format = "flac"
wav_output = "/content/so-vits-svc/results/" + wav_filename + "_" + trans + "key" + "_" + speaker + "." + wav_format

%cd /content/so-vits-svc
!python inference_main.py -n {wav_filename} -m {model_path} -s {speaker} -t {trans} -cr {cluster_infer_ratio} -c {config_path} -cm {kmeans_path} -sd {slice_db} -wf {wav_format} {apf} {fmp} {ehc}

download_after_inference = True

if download_after_inference:
  from google.colab import files
  files.download(wav_output)